In [741]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import sqrt 
from operator import itemgetter

In [742]:
#utils
def Sqr(a):
    return a*a
 
def Distance(x1,y1,x2,y2):
    return sqrt(Sqr(y2-y1)+Sqr(x2-x1))

In [743]:
fileToOpen = "p01.txt"

m = 0
n = 0
t = 0

In [744]:
ct = 0
with open(fileToOpen) as fin:
    rows = (line.split() for line in fin)
    for row in rows:
        if ct == 0:
            m = int(row[0])
            n = int(row[1])
            t = int(row[2])
            tmp = np.zeros([int(t),2])
            depotz = np.zeros([t, 5])
            customerz = np.zeros([n,5])
        elif ct > 0 and ct <= t:
            for depot in tmp:
                tmp[ct-1] = (row[0],row[1])
        elif ct >= t+1 and ct <= n+t:
            for element in range(n):
                a = n+t+1
                customerz[ct-a] = (row[0],row[1],row[2],row[3],row[4])
        elif ct > t+n:
            a = n+t+1
            for element in range(t):
                mdr = tmp[element][0]
                mlv = tmp[element][1]
                depotz[ct-a] = (row[0],row[1],row[2],mdr,mlv)
        ct = ct + 1
                
print (str(m) +" "+ str(n) +" "+ str(t) )

vehiclesMxW = depotz[0][4]
nbVehicles = 16 #np.random.randint(t, m*t)  #aka number of routes
indivPerGen = 100

4 50 4


In [745]:

print(depotz)
print(customerz)

ml = depotz[0][4]

[[51. 20. 20.  0. 80.]
 [52. 30. 40.  0. 80.]
 [53. 50. 30.  0. 80.]
 [54. 60. 50.  0. 80.]]
[[ 1. 37. 52.  0.  7.]
 [ 2. 49. 49.  0. 30.]
 [ 3. 52. 64.  0. 16.]
 [ 4. 20. 26.  0.  9.]
 [ 5. 40. 30.  0. 21.]
 [ 6. 21. 47.  0. 15.]
 [ 7. 17. 63.  0. 19.]
 [ 8. 31. 62.  0. 23.]
 [ 9. 52. 33.  0. 11.]
 [10. 51. 21.  0.  5.]
 [11. 42. 41.  0. 19.]
 [12. 31. 32.  0. 29.]
 [13.  5. 25.  0. 23.]
 [14. 12. 42.  0. 21.]
 [15. 36. 16.  0. 10.]
 [16. 52. 41.  0. 15.]
 [17. 27. 23.  0.  3.]
 [18. 17. 33.  0. 41.]
 [19. 13. 13.  0.  9.]
 [20. 57. 58.  0. 28.]
 [21. 62. 42.  0.  8.]
 [22. 42. 57.  0.  8.]
 [23. 16. 57.  0. 16.]
 [24.  8. 52.  0. 10.]
 [25.  7. 38.  0. 28.]
 [26. 27. 68.  0.  7.]
 [27. 30. 48.  0. 15.]
 [28. 43. 67.  0. 14.]
 [29. 58. 48.  0.  6.]
 [30. 58. 27.  0. 19.]
 [31. 37. 69.  0. 11.]
 [32. 38. 46.  0. 12.]
 [33. 46. 10.  0. 23.]
 [34. 61. 33.  0. 26.]
 [35. 62. 63.  0. 17.]
 [36. 63. 69.  0.  6.]
 [37. 32. 22.  0.  9.]
 [38. 45. 35.  0. 15.]
 [39. 59. 15.  0. 14.]
 [40.  5. 

In [746]:
#np.random.seed(0)

In [747]:
class Customer:
    def __init__(self,id,x,y,duration,demand):
        self.id = id
        self.x = x
        self.y = y 
        self.duration = duration
        self.demand = demand
        
            
    def __repr__(self):
        return "Client: " +str(int(self.id)) +" "+ str(int(self.x)) +" "+ str(int(self.y)) + " " + str(int(self.duration)) + " " + str(int(self.demand))

In [748]:
#Is a set of chromosomes
class Route:
    def __init__(self, ID, depot):
        self.id = ID
        self.depot = depot #depot assigned to this route
        self.customers = list() 
        self.capacity = 0 #total of the demand
        self.ttlDistance = 0 #Ttl distance of the road
        self.maxLoad = ml #maxLoad per vehicles
        self.oc = 0
        
    def getTotalDistanceR(self):
        ttl = 0
        vTotal = 0
        if len(self.customers) == 0:
            return 0
        first_cust = self.customers[0]
        last_cust = self.customers[-1]
        #calculating cost between each cust
        for c in range(len(self.customers)-1):
            #print("x: " +str(self.customers[c].x) +" y: " + str(self.customers[c].y))
            ttl = ttl + Distance(self.customers[c].x,self.customers[c].y,self.customers[c+1].x,self.customers[c+1].y)
        #and between depot at the begining and end of the route since the veh needs to stop where it started
        addtt = 0
        addtt = addtt + Distance(first_cust.x, first_cust.y, self.depot.x, self.depot.y)
        addtt = addtt + Distance(last_cust.x, last_cust.y, self.depot.x, self.depot.y)
        vTotal = ttl + addtt
                #print("Distance bw cust: " + str(ttl) + " Added to beg and end: " + str(addtt)+ " Total: " + str(vTotal))
        self.ttlDistance = vTotal
        #print('ttR'+ str( self.ttlDistance))
        return vTotal
    
    def getCapacity(self):
        return self.capacity
    
    #Check if the capacity is greater than allowed (reassign self too)
    def isOverloaded(self):
        ttl = 0
        for c in self.customers:
            ttl = ttl + c.demand
        self.capacity = ttl
        if ttl > self.maxLoad:
            self.oc = self.oc + 1
            return True
        else:
            return False
        
    def __repr__(self):
        return "Route: " +str(self.id) +" "+ str(self.depot) + " " + str(len(self.customers))+ " Sz:"+ str(len(self.customers)) + " C:" + str(self.capacity) + " T:" + str(self.ttlDistance)

In [749]:
#Is a set of genes
class Depot:
    def __init__(self, id, x, y): 
        self.id = id
        self.x = x
        self.y = y
        self.customers = list()
        self.maxVehicle = m
        self.routes = list()
        for nbR in range(self.maxVehicle):
            self.routes.append(Route(nbR, self))
        
    #assign a customer to a random route    
    def addCustomer(self,c):
        #Selecting which route to assign it to
        choice = np.random.randint(0,self.maxVehicle)
        #print( str(self.id)+ ": putting in:" + str(choice) + " " +str(c))
        self.routes[choice].customers.append(c)
    
    def getCapacity(self):
        ttlC = 0
        l = list()
        ro = list()
        for r in self.routes:
            ttlC = ttlC + r.capacity
            l.append(r.capacity)
            ro.append(r.oc)
        return ttlC, l, ro
    
    def getTotalDistanceD(self):
        distTotale = 0
        for r in self.routes:
            #we make the distance twice as great to avoid overloading
            if r.isOverloaded():
                distTotale = distTotale +r.getTotalDistanceR()*2
                #print("-" + str(distTotale))
            else:
                distTotale = distTotale +r.getTotalDistanceR()
                #print("+" + str(distTotale))
            
        """print(distTotale)
        print()"""
        return distTotale
    
    def __repr__(self):
        return "Depot: " + str(self.id) +" "  + str(self.x) +" "+ str(self.y) + " "+ str(len(self.routes) )

In [750]:
#Is a set od Depots representing a solution
class Individual:
    def __init__(self, id , gen):
        self.id = id
        self.gen = gen
        self.fitness = 9999
        self.depots = list()
        self.proba = 0 #chances for it to be picked as a parent
        
    def calculFitness(self):
        distTotale = 0
        for dpt in self.depots:
            distTotale = distTotale + dpt.getTotalDistanceD()
        self.fitness = distTotale

    def getTtlCapacity(self):
        ttlC = 0
        l = list()
        ovc = list()
        for d in self.depots:
            ttlC = ttlC + d.getCapacity()[0]
            l.append( d.getCapacity()[0])
        #overCapacity
        for oc in d.getCapacity()[2]:
            ovc.append(oc)
        return ttlC, l, ovc
        
    def addDepot(self,id, x, y):
        self.depots.append(Depot(id, x, y))
    
    
    def __repr__(self):
        return "Indiv:{}/{} {} {}".format(self.id, self.gen, str(int(self.fitness)), str(len(self.depots)))

In [751]:
def makeSolutionInit(i):
    #Making the t depots in the indiv
    for dpt in depotz:
        i.addDepot(dpt[0], dpt[1], dpt[2])
    #for each customer, assign them to the best depot
    for cli in customerz:
        bestDist = 99999
        closestDepot = None
        for testedDepot in i.depots:
            distToDepot = Distance(testedDepot.x, testedDepot.y, cli[1], cli[2])
            if distToDepot < bestDist:
                bestDist = distToDepot
                closestDepot = testedDepot
        closestDepot.addCustomer(Customer(cli[0], cli[1], cli[2], cli[3], cli[4]))
    return i

In [752]:
def makeGen0(individuals, ipg, epoch):
    while len(individuals) < ipg:
        i = Individual(len(individuals),epoch)
        i = makeSolutionInit(i)
        individuals.append(i)
        i.calculFitness()
    print(individuals)
    return individuals

In [753]:
#Given arr, an array , and index, a float this function return the right element from the assosiative array (used after normalization to chose a parent)
def probaPick(arr, index):
    #print(index)
    """print(index)"""
    if index < arr[0][0]: #checks if index given is small enough to be first value (avoid looping)
            return arr[0][1]
    if index > arr[len(arr)-1][0]: #Checks the last value (avoid loop)
        return arr[len(arr)-1][1]
    for el in range(len(arr)):
        #print(arr[el][0])
        if arr[el][0] <= index and arr[el+1][0] >= index:
            return arr[el][1] 

In [754]:
#next few lines are inspired by this : https://stackoverflow.com/questions/2772882/picking-a-random-item-based-on-probabilities
#normalized probability to have a greater chance to select the fitest candidate as a parent
def chooseParents(individuals):
    print("Selecting parents ...")
    listVal = list()
    size = 0
    for indi in individuals:
        size = size + 1
        listVal.append((indi.fitness, indi))
    """print(listVal)
    print()"""
    #sort array by value not indi
    listVal.sort(key=itemgetter(0), reverse=True)
    #assign probability depending of where in the list
    ct = 1
    for el in range(len(listVal)):
        indi = listVal[el][1]
        listVal[el] = [ct, indi]
        ct = ct + 1
    """print(listVal)
    print()"""
    #calculate cumulative proba
    lastList = list()
    cumul = 1
    for el in range(len(listVal)):
        if el == 0:
            listVal[el][0] = 1
        else:
            indi = listVal[el][1]
            cumul = listVal[el][0] + cumul
            listVal[el] = (cumul, indi)
    #normalize the probabilities
    """print(listVal)"""
    for el in range(len(listVal)):
        listVal[el] = (listVal[el][0]/cumul*100,listVal[el][1])
    """print(listVal)"""
    #chose the two parents    
    choice = np.random.uniform(0,100)
    parent1 = probaPick(listVal, choice)
    choice = np.random.uniform(0,100)
    parent2 = probaPick(listVal, choice)
    #Must make sure to not have the same parents
    while parent1.id ==parent2.id: #keep changing parent2 until it's not the sampe as parent1
        print("Same parent rerolling ... parent2")
        choice = np.random.uniform(0,100)
        parent2= probaPick(listVal, choice)
    
    return parent1, parent2

In [1402]:
# Recombines two individuals (parents) to produce two new individuals (offspring),
# by applying Best Cost Route Crossover:
# 1. Randomly select depot to undergo reproduction
# 2. Randomly select a route from each parent
# 3. For each selected route, remove all customers in that route from the other parent
# 4. For each customer c that was removed from parent p, insert customer c back in parent p at best location.
def crossover(parent1, parent2):
    
    choiceDepot = np.random.randint(0,t)
    depot1 = parent1.depots[choiceDepot]
    choiceRoute = np.random.randint(0,m)
    route1 = depot1.routes[choiceRoute]
    
    choiceDepot = np.random.randint(0,t)
    choiceRoute = np.random.randint(0,m)
    depot2 = parent2.depots[choiceDepot]
    route2 = depot2.routes[choiceRoute]
    
    removedCust1 = list()
    removedCust2 = list()
    
    #Check if the customer is in the other parent
    for c1 in route1.customers:
        for c2 in route2.customers:
            if c1.id == c2.id:
                print(len(route2.customers))
                print(c2.id)
                removedCust2.append(c2)
                route2.customers.remove(c2)
                for cc in route2.customers:
                    print(cc.id)
        print("r2c" + str(len(route2.customers)))
    
    for el in removedCust2:
        bestCost = 99999
        bestPlace = 0
        for i in range(len(route2.customers)):
            
    
    """#get best place to put customer back
    for cust in removedCust2:
        bestCost = 99999
        bestPlace = 0
        #For each deleted customer
        for i in range(len(route2.customers)):
            #If creating new route cost = d(c, depot)*2
            if len(route2.customers) == 0:
                cost = (Distance(cust.x, cust.y, depot2.x, depot2.y))*2
            #If at the begining or end of the list, cost = d(depot, c) + d(c, c(n-+1)) - d(c(n-+1), depot)
            print("it: " + str(i) )
            if i == 0 :
                if len(route2.customers) == 1:
                    cost = (Distance(cust.x, cust.y, depot2.x, depot2.y)) + Distance(cust.x, cust.y, route2.customers[1].x, route2.customers[1].y) - Distance(depot2.x, depot2.y, route2.customers[1].x, route2.customers[1].y) 
                    if cost < bestCost :
                        bestCost = cost
                        bestPlace = i
            elif i == len(route2.customers)-1:
                cost = (Distance(cust.x, cust.y, depot2.x, depot2.y)) + Distance(cust.x, cust.y, route2.customers[len(route2.customers)-1].x, route2.customers[len(route2.customers)-1].y) - Distance(depot2.x, depot2.y, route2.customers[len(route2.customers)-1].x, route2.customers[len(route2.customers)-1].y) 
                if cost < bestCost :
                        bestCost = cost
                        bestPlace = i
            else: #cost = d from cust to both cust - dist(ac1, ac2)
                cost = (Distance(cust.x, cust.y,  route2.customers[i].x,  route2.customers[i].y))+ (Distance(cust.x, cust.y,  route2.customers[i+1].x,  route2.customers[i+1].y)) 
                cost = cost - (Distance(route2.customers[i].x,  route2.customers[i].y,  route2.customers[i+1].x,  route2.customers[i+1].y))
                if cost < bestCost :
                        bestCost = cost
                        bestPlace = i
            print("Cost at {}: {}/{} bp:{}".format(i, cost, bestCost, bestPlace))
        route2.customers.insert(bestPlace, cust)
        print("r2b" + str(len(route2.customers)))"""
    return parent2
                
    
    """
    print(len(route1.customers))
    for c in route1.customers:
        removedCust1.append(c)
    route1.customers.clear()
    print(len(route2.customers))
    for c in route2.customers:
        removedCust2.append(c)
    route2.customers.clear()"""

    """print(choiceDepot)
    print(choiceRoute)"""
    print(route1)
    print(route2)
    print(removedCust1)
    print(removedCust2)

In [1403]:
def mainProg(epochs=10, indivPerGen=20):  
    individuals = list()
    individualsNextGen = list()
    #Epoch is a generation
    for epoch in range(epochs):
        print("Generation: " + str(epoch))
        if epoch == 0:
            individualsNextGen = makeGen0(individuals, indivPerGen, epoch)
        else:
            #Main loop
            #We need to select 2 parents (depending of their fitness)
            print(individuals)
            
            while(len(individualsNextGen) < indivPerGen ):
                parent1 , parent2 = chooseParents(individuals)
                print(parent1)
                print(parent2)
                #Cross their genes
                child1 = crossover(parent1, parent2)
                child2 = crossover(parent2, parent1)

                print(child1)
                print(child2)

                individualsNextGen.append(child1)
                individualsNextGen.append(child2)
            #Give a chance to mutate
            for i in individualsNextGen:
                i.calculFitness()
        print("--------------")
        individuals = individualsNextGen.copy()
        individualsNextGen.clear()
        
        
                
                
        """ttl = 0
        for i in individuals:
            for d in i.depots:
                #print(d)
                print(d)
                for r in d.routes:
                    #ttl = ttl + len(r.customers)
                    print((r.id))
                    print(r.oc)
                    print(r.capacity)
            print("Indiv" + str(i))
        print(ttl)"""

In [1404]:
individuals = list()
individualsNextGen = list()
mainProg()

Generation: 0
[Indiv:0/0 1438 4, Indiv:1/0 1149 4, Indiv:2/0 1525 4, Indiv:3/0 1231 4, Indiv:4/0 1146 4, Indiv:5/0 1620 4, Indiv:6/0 1388 4, Indiv:7/0 1048 4, Indiv:8/0 1343 4, Indiv:9/0 1338 4, Indiv:10/0 1223 4, Indiv:11/0 1389 4, Indiv:12/0 1332 4, Indiv:13/0 1279 4, Indiv:14/0 1232 4, Indiv:15/0 1526 4, Indiv:16/0 1461 4, Indiv:17/0 1396 4, Indiv:18/0 1424 4, Indiv:19/0 1263 4]
--------------
Generation: 1
[Indiv:0/0 1438 4, Indiv:1/0 1149 4, Indiv:2/0 1525 4, Indiv:3/0 1231 4, Indiv:4/0 1146 4, Indiv:5/0 1620 4, Indiv:6/0 1388 4, Indiv:7/0 1048 4, Indiv:8/0 1343 4, Indiv:9/0 1338 4, Indiv:10/0 1223 4, Indiv:11/0 1389 4, Indiv:12/0 1332 4, Indiv:13/0 1279 4, Indiv:14/0 1232 4, Indiv:15/0 1526 4, Indiv:16/0 1461 4, Indiv:17/0 1396 4, Indiv:18/0 1424 4, Indiv:19/0 1263 4]
Selecting parents ...
Indiv:16/0 1461 4
Indiv:3/0 1231 4
r2c7
r2c7
7
43.0
8.0
11.0
12.0
27.0
31.0
47.0
r2c6
it: 0


UnboundLocalError: local variable 'cost' referenced before assignment

In [839]:
a = Customer(1,10,10,0,0)
al = list([a])
print(a in al)

True
